In [57]:
import requests
import json
import time
import pandas as pd

# Token accessToken 及权限校验机制
getAccessTokenUrl = 'https://quantapi.51ifind.com/api/v1/get_access_token'
# 获取refresh_token需下载Windows版本接口包解压，打开超级命令-工具-refresh_token查询
with open ('token.txt', 'r') as file:
    refreshtoken = file.read().strip()
getAccessTokenHeader = {"Content-Type": "application/json", "refresh_token": refreshtoken}
getAccessTokenResponse = requests.post(url=getAccessTokenUrl, headers=getAccessTokenHeader)
accessToken = json.loads(getAccessTokenResponse.content)['data']['access_token']
print(accessToken)

thsHeaders = {"Content-Type": "application/json", "access_token": accessToken}

4222a95b2ee94a2b1910d968467947b745562266.signs_NzMwMTMyNTUw


In [58]:
thsUrl = 'https://quantapi.51ifind.com/api/v1/cmd_history_quotation'
thsPara = {"codes":
               "000001.SZ,600000.SH",
           "indicators":
               "open,high,low,close",
           "startdate":
               "2021-07-05",
           "enddate":
               "2022-07-05",
           "functionpara":
               {"Fill": "Blank"}
           }
thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)

In [60]:
thsResponse.content

b'{"errorcode":0,"errmsg":"","tables":[{"thscode":"000001.SZ","time":["2021-07-05","2021-07-06","2021-07-07","2021-07-08","2021-07-09","2021-07-12","2021-07-13","2021-07-14","2021-07-15","2021-07-16","2021-07-19","2021-07-20","2021-07-21","2021-07-22","2021-07-23","2021-07-26","2021-07-27","2021-07-28","2021-07-29","2021-07-30","2021-08-02","2021-08-03","2021-08-04","2021-08-05","2021-08-06","2021-08-09","2021-08-10","2021-08-11","2021-08-12","2021-08-13","2021-08-16","2021-08-17","2021-08-18","2021-08-19","2021-08-20","2021-08-23","2021-08-24","2021-08-25","2021-08-26","2021-08-27","2021-08-30","2021-08-31","2021-09-01","2021-09-02","2021-09-03","2021-09-06","2021-09-07","2021-09-08","2021-09-09","2021-09-10","2021-09-13","2021-09-14","2021-09-15","2021-09-16","2021-09-17","2021-09-22","2021-09-23","2021-09-24","2021-09-27","2021-09-28","2021-09-29","2021-09-30","2021-10-08","2021-10-11","2021-10-12","2021-10-13","2021-10-14","2021-10-15","2021-10-18","2021-10-19","2021-10-20","2021-1